In [1]:
import os
os.chdir("..")

##### Import Libraries

In [36]:
import sys
sys.path.append("DANBO-pytorch")

import torch
import numpy as np
from glob import glob
import plotly.io as pio
from tqdm import tqdm, trange

import core.utils.skeletons as skeleton
from core.load_data import generate_bullet_time
from core.utils.extras import load_pickle, alpha_to_hip_1st
from core.utils.skeleton_utils import plot_skeleton3d as danbo_plot_skeleton3d
from core.utils.skeleton_utils import get_kp_bounding_cylinder, plot_bounding_cylinder

pio.renderers.default = 'notebook' 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Load reconstructed 3D data - (replace with your own id)

In [38]:
seq_name = "Subj3"
# add reconstruction identifier below. Can be found in 'outputs/metrics.txt' e.g xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx_cam_0
comb = "cdaec2b9-6f15-45a1-8c25-778ce83089d9_cam_0" # replace with your own id here for e.g subject 3
iterations = 1999  # also encoded on the reconstruction result pickle file name

id = comb.split("_cam_")[0]
view = cam = comb.split("_cam_")[1]
files = sorted(glob(f"outputs/recon_results_no_gt2d_no_gtfocal/{view}/*{id}.pickle"))

kp3d, kp3d_h, proj2d_real, proj2d_virt, img_urls, v_view_h, r_view_h = [], [], [], [], [], [], []
est_2d_real, est_2d_virt, rotation, theta, bf_build = [], [], [], [], []
bone_orient_store, k_optim, bone_length = [], [], []
p_dash_2d, p_2d, N_2d, normal_end_2d = [],[],[],[]
ground_end_2d, refine_ground_end_2d, otho_end_2d = [],[], []
l2ws, bf_positive, m_normal, g_normal = [], [], [], []
chosen_frames, flipped_frames = [], []
N3d, p3d, p_dash3d, otho = [],[],[], []
real_feet_mask, virt_feet_mask = [], []
A, A_dash, A_dash_tuple = [], [], []
avg_D, plane_d = [], []

# gather reconstructions 
for i in range(len(files)):
    i = iterations
    
    filename = f"outputs/recon_results_no_gt2d_no_gtfocal/{view}/{seq_name}_{view}_{i}_{i+1}iter_"+"{'A_dash': False, 'A_dash_fewer': False, 'K_op': True, 'bf_op': True, 'bf_build': True, 'n_g_single': True, 'n_m_single': True}_"+f"{id}.pickle"
    from_pickle = load_pickle(filename)
    
    kp3d.extend(from_pickle["kp3d"]) 
    proj2d_real.extend(from_pickle["proj2d_real"])
    proj2d_virt.extend(from_pickle["proj2d_virt"])
    est_2d_real.extend(from_pickle["est_2d_real"])
    est_2d_virt.extend(from_pickle["est_2d_virt"])
    img_urls.extend(from_pickle["img_urls"])
    kp3d_h.extend(from_pickle["kp3d_h"])
    v_view_h.extend(from_pickle["v_view_h"])
    r_view_h.extend(from_pickle["r_view_h"])
    rotation.extend(from_pickle["optim_rotation3x3"])
    theta.extend(from_pickle["optim_theta"]) 
    bf_build.extend(from_pickle["bf_build"])
    bone_orient_store.extend(from_pickle["b_orientation"])
    k_optim.extend(from_pickle["K_optim"])
    bf_positive.extend(from_pickle["bf_positive"])
    m_normal.extend(from_pickle["n_m"])
    g_normal.extend(from_pickle["n_g_mini"])
    otho.extend(from_pickle["otho"])
    N3d.extend(from_pickle["N3d"])
    p3d.extend(from_pickle["p3d"])
    p_dash3d.extend(from_pickle["p_dash3d"])
    flipped_frames.extend(from_pickle["flipped_frames"])
    A.extend(from_pickle["final_A"])
    A_dash.extend(from_pickle["final_A_dash"])
    avg_D.extend(from_pickle["avg_D"])
    plane_d.extend(from_pickle["plane_d"])
    
    p_dash_2d.extend(from_pickle["p_dash_2d"]); p_2d.extend(from_pickle["p_2d"]); N_2d.extend(from_pickle["N_2d"]);
    normal_end_2d.extend(from_pickle["normal_end_2d"]); ground_end_2d.extend(from_pickle["ground_end_2d"])
    refine_ground_end_2d.extend(from_pickle["refine_ground_end_2d"]); otho_end_2d.extend(from_pickle["otho_end_2d"])
    l2ws.extend(from_pickle["l2ws"])

kp3d = torch.stack(kp3d)
kp3d_h = torch.stack(kp3d_h)
v_view_h = torch.stack(v_view_h)
r_view_h = torch.stack(r_view_h)
rotation_optim = torch.stack(rotation)
theta_optim = torch.stack(theta)
bf_build = torch.stack(bf_build)
bone_orient_store = torch.stack(bone_orient_store)
k_optim = torch.stack(k_optim).view(-1,3,3)
bf_positive = torch.stack(bf_positive)
A = torch.stack(A)
A_dash = torch.stack(A_dash)
avg_D = torch.stack(avg_D)
plane_d = torch.stack(plane_d)

chosen_frames = from_pickle["chosen_frames"]
initial_pose3d = from_pickle["initial_pose3d"]

p_dash_2d, p_2d = torch.stack(p_dash_2d), torch.stack(p_2d)
N_2d, normal_end_2d = torch.stack(N_2d), torch.stack(normal_end_2d)
ground_end_2d = torch.stack(ground_end_2d)
refine_ground_end_2d = torch.stack(refine_ground_end_2d)
otho_end_2d = torch.stack(otho_end_2d)
l2ws = torch.stack(l2ws)
m_normal = torch.stack(m_normal)
g_normal = torch.stack(g_normal)
N3d = torch.stack(N3d)
p3d = torch.stack(p3d)
otho = torch.stack(otho)
p_dash3d = torch.stack(p_dash3d)

proj2d_real = torch.stack(proj2d_real)
proj2d_virt = torch.stack(proj2d_virt)
est_2d_real = torch.stack(est_2d_real)
est_2d_virt = torch.stack(est_2d_virt)

proj2d_real.shape, proj2d_virt.shape, est_2d_real.shape, theta_optim.shape, rotation_optim.shape, \
bf_build.shape, bone_orient_store.shape, k_optim.shape, len(img_urls)


(torch.Size([2000, 25, 2]),
 torch.Size([2000, 25, 2]),
 torch.Size([2000, 25, 2]),
 torch.Size([2000, 26, 6]),
 torch.Size([2000, 26, 3, 3]),
 torch.Size([1, 25, 1, 1]),
 torch.Size([2000, 26, 3, 3]),
 torch.Size([1, 3, 3]),
 2000)

##### extra accesories

In [39]:
add_top = False

if seq_name in ["Subj3"]:
    add_top = True # for Tall people with longer head distance, e.g Subj3 
d_size = 5
add_top

True

In [40]:
if add_top: # x2 
    head_margin = 80; foot_margin = 40; side_margin = 80
else:
    head_margin = 40; foot_margin = 20; side_margin = 15
head_margin, foot_margin, side_margin

(80, 40, 80)

##### Last data stamp 

In [41]:
# timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H") # uncomment 
timestamp = '2023-12-21-14' # please comment this out for real-timestamp
timestamp

'2023-12-21-14'

In [42]:
cam, comb, timestamp

('0', 'cdaec2b9-6f15-45a1-8c25-778ce83089d9_cam_0', '2023-12-21-14')

##### Pose

In [43]:
kp3d_h_hipfirst = alpha_to_hip_1st(kp3d_h).detach().numpy() # same as real view

v_view_h_hipfirst = alpha_to_hip_1st(v_view_h).detach().numpy()
r_view_h_hipfirst = alpha_to_hip_1st(r_view_h).detach().numpy()
B=v_view_h_hipfirst.shape[0]
B

2000

In [44]:
ratio = 0.0
test_set = int(B * ratio)
train_set = B-test_set
train_set, test_set

(2000, 0)

In [45]:
poseid = 0 #150

##### create cylinders 

In [46]:
# first convert to hip_first
if add_top:
    top_expand_ratio=3.
else:
    top_expand_ratio=1.
    if seq_name == "Daniel_normal" or seq_name == "Chunjin":
        top_expand_ratio=1.5

extend_mm=250
# global pose
v_cylinder_params = get_kp_bounding_cylinder(v_view_h_hipfirst,
                                               skel_type=skeleton.CMUSkeleton, extend_mm=extend_mm,
                                               top_expand_ratio=top_expand_ratio,
                                               head='-y')
v_cylinder_params.shape, v_cylinder_params[poseid].min(), add_top, top_expand_ratio

Head direction: -y


((2000, 5), -5.0286098, True, 3.0)

##### Create Betas and camera (same for all cams?)

In [47]:
def get_c2ws_demo():
    """unflipped as in A-NeRF"""
    c2ws_real = torch.Tensor([[1,0,0,0],
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]])
    return c2ws_real

##### Configure settings: e.g add good top_margin_ratio

In [48]:
top_view = False
generate_motion = True
n_bullet = 90   
clockwise = True
bullet_ang=360

In [49]:
if generate_motion:
    c2ws_real_temp = get_c2ws_demo().detach().numpy().astype(np.float32)
    motion_c2ws = generate_bullet_time(c2ws_real_temp, n_views=n_bullet, bullet_ang=bullet_ang).reshape(-1, 4, 4)
    # motion_c2ws = generate_bullet_time(c2ws_real_temp, n_views=n_bullet, bullet_ang=bullet_ang).transpose(1, 0, 2, 3).reshape(-1, 4, 4)
    # back to tensor
    motion_c2ws_tensor = torch.tensor(motion_c2ws)
motion_c2ws_tensor.shape

torch.Size([90, 4, 4])

In [50]:
def rotate_kps(motion_c2ws_tensor, n_bullet):
    kps3d = torch.tensor(v_view_h_hipfirst[poseid:poseid+1])
    kps_homo = torch.cat((kps3d, kps3d.new_ones(1).expand(*kps3d.shape[:-1], 1)), 2)
    rotated_kps3d = torch.bmm(kps_homo.repeat(n_bullet,1,1), motion_c2ws_tensor).detach().numpy().astype(np.float32)
    return rotated_kps3d

rotated_kps3d = rotate_kps(motion_c2ws_tensor, n_bullet)
rotated_kps3d.shape

(90, 26, 4)

In [51]:
fig_path = f"outputs/rotation/top_view_{top_view}/poseid_{poseid}"
fig_dir = f"{fig_path}/{cam}/{comb}/{timestamp}"
os.makedirs(fig_dir, exist_ok=True)

##### 3D rotation
###### ref: https://community.plotly.com/t/rotating-3d-plots-with-plotly/34776/2

In [ ]:
print(f"print {top_view}")

if top_view:
    zoom_x=1; zoom_y=2; zoom_z=1
    cam_eye_loc=[0,-1,0]
    cam_up_orient=[1,1,0]

else: # side_view
    zoom_x=1; zoom_y=1; zoom_z=2
    cam_eye_loc=[0,0,1.2]
    cam_up_orient=[0,0,0]

comb_idxs = np.concatenate([np.arange(45,90), np.arange(0,45)])
for iter_, motion_idx in tqdm(enumerate(comb_idxs)):
    fig = None
    
    fig= plot_bounding_cylinder(rotated_kps3d[motion_idx,:,:3], fig=fig, head="-y")
    fig= danbo_plot_skeleton3d(rotated_kps3d[motion_idx,:,:3], fig=fig,line_width=2,\
                          visible_x=False, visible_y=False, visible_z=False, \
                          cam_eye_loc=cam_eye_loc, zoom_x=zoom_x, zoom_y=zoom_y, zoom_z=zoom_z,
                          cam_up_orient=cam_up_orient)
    fig.update_layout(
        scene = dict(
            xaxis = dict(showticklabels=False),
            yaxis = dict(showticklabels=False),
            zaxis =dict(showticklabels=False)
            ))
    fig.write_image(f"{fig_dir}/fig_{iter_:04d}.png")
    # fig.show()

In [54]:
save_path = "outputs/gif"
to_vid = f"ffmpeg -framerate 50 -pattern_type glob -i '{fig_dir}/fig*.png' -vf 'pad=ceil(iw/2)*2:ceil(ih/2)*2' -y -c:v libx264 -r 30 -pix_fmt yuv420p '{save_path}/3D_rotation.mp4'"
to_gif = f"ffmpeg -i '{save_path}/3D_rotation.mp4' -pix_fmt yuv420p -loop 0 '{save_path}/3D_rotation.gif'"
os.system(to_vid)
os.system(to_gif)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

256

##### single frame skeleton

In [ ]:
fig = None
fig= danbo_plot_skeleton3d(v_view_h_hipfirst[poseid], fig=fig,line_width=2,\
                              visible_x=False, visible_y=False, visible_z=False, \
                              cam_eye_loc=cam_eye_loc, zoom_x=zoom_x, zoom_y=zoom_y, zoom_z=zoom_z,
                              cam_up_orient=cam_up_orient)
fig.update_layout(
    scene = dict(
        xaxis = dict(showticklabels=False),
        yaxis = dict(showticklabels=False),
        zaxis =dict(showticklabels=False)
        )
    )
fig.show()